In [1]:
from datetime import datetime
import asyncio
from ib_insync import *
import nest_asyncio
nest_asyncio.apply()
util.startLoop()
ib = IB()
ib.connect('host.docker.internal', 7497, clientId=15)


<IB connected to host.docker.internal:7497 clientId=15>

In [ ]:
from datetime import datetime
import pandas as pd
import asyncio
from ib_insync import *
import nest_asyncio
import logging,sys
import functools
# from asgiref.sync import sync_to_async


def histBar(contract_d,param_d,useRTH=False):
    try:
        util.startLoop()
        print('starting')
        df = pd.DataFrame()
        contract = Stock(contract_d['symbol'],contract_d['exchange'], contract_d['currency'])
        bars = ib.reqHistoricalData(contract,endDateTime=param_d['endt'], durationStr=param_d['dur'],barSizeSetting=param_d['candles'], whatToShow=param_d['show'], useRTH=useRTH)
        df = util.df(bars)
        df = df.set_index('date')
        df.columns = map(str.lower, df.columns)
    except Exception as e: print(e)
    finally:
        print('ended')
        return df

    

from concurrent.futures import ThreadPoolExecutor
from aiokafka import AIOKafkaConsumer
import json


async def consume():
    global q
    consumer = AIOKafkaConsumer(
        'test_req',
        loop=loop, bootstrap_servers='broker:29092',
        group_id="my-group")
    await consumer.start()
    try:
        async for msg in consumer:
            print("consumed: ", msg.topic, msg.partition, msg.offset,
                  msg.key, msg.value, msg.timestamp)
            key = msg.key.decode('UTF-8')
            data = json.loads(msg.value)
            q[key] = {}
            q[key]['data'] = data
            q[key]['status'] = 'accepted'
    finally:
        await consumer.stop()

async def control():
    global q
    global fc
    while True:
        await asyncio.sleep(3)
        print(f'slept ----------- {datetime.now()}')
        for key,val in q.items():
            print(key,q[key]['status'])
            if fc > 2: pass
            else:
                if q[key]['status']=='accepted':
                    q[key]['status']='started'
                    fc +=1
#                     print(q[key])
                    try:result = await histBar(q[key]['data']['contract'],q[key]['data']['params'])
                    except Exception as e: print(e)
                    
# loop.run_until_complete(control())
# loop.run_until_complete(consume())

if __name__ == '__main__':
    logging.getLogger("asyncio").setLevel(logging.INFO)
    logging.basicConfig(
    level=logging.INFO,
    format='%(threadName)10s %(name)18s: %(message)s',
    stream=sys.stderr,)

    nest_asyncio.apply()
    
    util.startLoop()
    ib = IB()
    ib.connect('host.docker.internal', 7497, clientId=15)
    
    loop = asyncio.get_event_loop()
    loop.set_debug(enabled=True)
    
    q = {}
    fc = 0

    
    try:
        consume = asyncio.create_task(consume())
        control = await asyncio.create_task(control())
        loop.run_forever()
    except Exception as e: 
#         loop.stop()
        print("FATAL: ",e)
    finally:loop.close()

MainThread   ib_insync.client: Connecting to host.docker.internal:7497 with clientId 15...
MainThread   ib_insync.client: Connected
MainThread   ib_insync.client: Logged on to server version 152
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfuture
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfarm.nj
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfarm
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:euhmds
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:ushmds.nj
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:fundfarm
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:ushmds
MainThread  ib_insync.wrapper: Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
MainThread   ib_insync.client: A

slept ----------- 2021-02-28 00:48:09.043439
slept ----------- 2021-02-28 00:48:12.044394
slept ----------- 2021-02-28 00:48:15.045465
slept ----------- 2021-02-28 00:48:18.046743
slept ----------- 2021-02-28 00:48:21.048442
slept ----------- 2021-02-28 00:48:24.049789
slept ----------- 2021-02-28 00:48:27.051896
slept ----------- 2021-02-28 00:48:30.053230
slept ----------- 2021-02-28 00:48:33.054816
slept ----------- 2021-02-28 00:48:36.056829
slept ----------- 2021-02-28 00:48:39.057900
slept ----------- 2021-02-28 00:48:42.059757
slept ----------- 2021-02-28 00:48:45.061057
slept ----------- 2021-02-28 00:48:48.062634
slept ----------- 2021-02-28 00:48:51.063709
slept ----------- 2021-02-28 00:48:54.064908
slept ----------- 2021-02-28 00:48:57.066676
slept ----------- 2021-02-28 00:49:00.068757
slept ----------- 2021-02-28 00:49:03.071480
slept ----------- 2021-02-28 00:49:06.074099
slept ----------- 2021-02-28 00:49:09.076630
slept ----------- 2021-02-28 00:49:12.078235
slept ----

slept ----------- 2021-02-28 00:57:18.357437
slept ----------- 2021-02-28 00:57:21.358547
slept ----------- 2021-02-28 00:57:24.359775
slept ----------- 2021-02-28 00:57:27.360590
slept ----------- 2021-02-28 00:57:30.362296
slept ----------- 2021-02-28 00:57:33.364014
slept ----------- 2021-02-28 00:57:36.364754
slept ----------- 2021-02-28 00:57:39.366272
slept ----------- 2021-02-28 00:57:42.368590
slept ----------- 2021-02-28 00:57:45.370892
slept ----------- 2021-02-28 00:57:48.372308
slept ----------- 2021-02-28 00:57:51.372992
slept ----------- 2021-02-28 00:57:54.374113
slept ----------- 2021-02-28 00:57:57.375565
slept ----------- 2021-02-28 00:58:00.377191
slept ----------- 2021-02-28 00:58:03.379032
slept ----------- 2021-02-28 00:58:06.379979
slept ----------- 2021-02-28 00:58:09.380535
slept ----------- 2021-02-28 00:58:12.381913
slept ----------- 2021-02-28 00:58:15.383032
slept ----------- 2021-02-28 00:58:18.384520
slept ----------- 2021-02-28 00:58:21.385697
slept ----

In [3]:
from confluent_kafka import Producer, KafkaError, Consumer
import json
import socket
import uuid
v = {'contract':['STOCK','SPY','SMART','USD'],'params':{'endt':'20201005 23:99','dur':'1 D','candles':'1 mins','show':'TRADES'}}
conf = {'bootstrap.servers': "broker:29092",
        'client.id': socket.gethostname()}
producer = Producer(conf)
producer.produce('test_req', key=str(uuid.uuid4()), value=json.dumps(v))

<ipython-input-3-16429829fb05>:9: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  producer.produce('test_req', key=str(uuid.uuid4()), value=json.dumps(v))


In [4]:
from aiokafka import AIOKafkaConsumer

loop = asyncio.get_event_loop()
q = []
async def consume():
    consumer = AIOKafkaConsumer(
        'test_req',
        loop=loop, bootstrap_servers='broker:29092',
        group_id="my-group")
    # Get cluster layout and join group `my-group`
    await consumer.start()
    try:
        # Consume messages
        async for msg in consumer:
            print("consumed: ", msg.topic, msg.partition, msg.offset,
                  msg.key, msg.value, msg.timestamp)
            key = msg.key.decode('UTF-8')
            data = json.loads(msg.value)
            data['uuid'] = key
            data['status'] = 'accepted'
            q.append(data)
    finally:
        # Will leave consumer group; perform autocommit if enabled.
        await consumer.stop()

loop.run_until_complete(consume())

consumed:  test_req 0 7 b'c571a9df-f531-4805-8755-c9f5bdd1d665' b'{"contract": ["STOCK", "SPY", "SMART", "USD"], "params": {"endt": "", "dur": "", "candles": "1 mins", "show": "TRADES"}}' 1601950763182


{'contract': ['STOCK', 'SPY', 'SMART', 'USD'],
 'params': {'endt': '', 'dur': '', 'candles': '1 mins', 'show': 'TRADES'}}

consumed:  test_req 0 8 b'38934c7c-ad88-43ee-bcca-c9829e1efdd3' b'{"contract": ["STOCK", "SPY", "SMART", "USD"], "params": {"endt": "", "dur": "", "candles": "1 mins", "show": "TRADES"}}' 1601950763409


{'contract': ['STOCK', 'SPY', 'SMART', 'USD'],
 'params': {'endt': '', 'dur': '', 'candles': '1 mins', 'show': 'TRADES'}}

KeyboardInterrupt: 

In [7]:
loop = asyncio.get_event_loop()
q = []
fc = 0
async def control():
    global fc
    while True:
        await asyncio.sleep(3)
        print('slept')
        if fc > 3: 
            print('maxed',fc)
            print(q)
        else:
            for r in q:
                if r['status']=='accepted':
                    r['status']=='started'
                    fc +=1
                    print('started 1')
                    
loop.run_until_complete(control())

slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept
slept


KeyboardInterrupt: 

In [10]:
req = {}


async def histBar(contract,endt,dStr,bSize,show,useRTH=False):
    bars = ib.reqHistoricalData(contract,endDateTime=end, durationStr=dStr,barSizeSetting=bSize, whatToShow=show, useRTH=useRTH)
    df = util.df(bars)
    df = df.set_index('date')
    df.columns = map(str.lower, df.columns)
    return df

async def main():
    while True:
        asyncio.sleep(.1)
        contract = Stock('SPY', 'SMART', 'USD')
        df = await histBar(contract,end,"1 W","1 min","TRADES")
        display(df)
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


,open,high,low,close,volume,average,barcount
date,,,,,,,
2020-07-30 04:00:00,323.00,323.00,322.59,322.80,126,322.918,28
2020-07-30 04:01:00,322.75,322.81,322.75,322.81,4,322.795,2
2020-07-30 04:02:00,322.80,322.86,322.80,322.84,38,322.813,12
2020-07-30 04:03:00,322.90,322.90,322.89,322.89,13,322.898,5
2020-07-30 04:04:00,322.90,322.92,322.90,322.92,12,322.903,3
...,...,...,...,...,...,...,...
2020-08-28 11:37:00,349.20,349.20,349.07,349.08,419,349.145,281
2020-08-28 11:38:00,349.08,349.21,349.05,349.20,241,349.113,163
2020-08-28 11:39:00,349.20,349.26,349.12,349.14,305,349.207,176


In [2]:
import qgrid
import functools
from collections import namedtuple
import pandas as pd

pd.set_option('display.max_colwidth',100)
start = ''
end = "20200828 11:41:41"
contract = Stock('SPY', 'SMART', 'USD')

ticks = ib.reqHistoricalTicks(contract, start, end, 1000, 'TRADES', useRth=False)
df = util.df(ticks)
df = df.set_index('time')
df = df.tz_convert('America/Toronto')

df['pastLimit'] = [row.pastLimit for row in df['tickAttribLast']]
df['unreported'] = [row.unreported for row in df['tickAttribLast']]
del df['tickAttribLast']
display(df)
ticks = ib.reqHistoricalTicks(contract, start, end, 1000, 'BID_ASK', useRth=False)
df = util.df(ticks)
df = df.set_index('time')
df = df.tz_convert('America/Toronto')
# display(df)
# print(df.dtypes)

df['bidPastLow'] = [row.bidPastLow for row in df['tickAttribBidAsk']]
df['askPastHigh'] = [row.askPastHigh for row in df['tickAttribBidAsk']]
del df['tickAttribBidAsk']
display(df)
# df.to_csv("tsla.csv")


end = "20200828 23:59:00"
bars = ib.reqHistoricalData(
    contract,endDateTime=end, durationStr='1 D',
    barSizeSetting='5 mins', whatToShow='TRADES', useRTH=False)
df = util.df(bars)
df = df.set_index('date')
display(df)

bars = ib.reqHistoricalData(
    contract,endDateTime=end, durationStr='1 D',
    barSizeSetting='5 mins', whatToShow='BID_ASK', useRTH=False)
df = util.df(bars)
df = df.set_index('date')
display(df)
bars = ib.reqHistoricalData(
    contract,endDateTime=end, durationStr='1 D',
    barSizeSetting='5 mins', whatToShow='MIDPOINT', useRTH=False)

df = util.df(bars)
df = df.set_index('date')

display(df)

##AZURE COSMOS DB or MONGO

,price,size,exchange,specialConditions,pastLimit,unreported
time,,,,,,
2020-08-28 11:38:51-04:00,349.1900,2,ISLAND,I,False,True
2020-08-28 11:38:51-04:00,349.2000,4,ISLAND,F I,False,True
2020-08-28 11:38:51-04:00,349.1975,4,FINRA,I,False,True
2020-08-28 11:38:51-04:00,349.2000,20,FINRA,I,False,True
2020-08-28 11:38:51-04:00,349.2000,7,ISLAND,I,False,True
...,...,...,...,...,...,...
2020-08-28 11:41:39-04:00,349.1000,100,BATS,F,False,False
2020-08-28 11:41:39-04:00,349.1000,24,ISLAND,F I,False,True
2020-08-28 11:41:39-04:00,349.1001,1,FINRA,I,False,True


,priceBid,priceAsk,sizeBid,sizeAsk,bidPastLow,askPastHigh
time,,,,,,
2020-08-28 11:41:36-04:00,349.12,349.13,100,2500,False,False
2020-08-28 11:41:37-04:00,349.12,349.13,300,2500,False,False
2020-08-28 11:41:37-04:00,349.12,349.13,300,2600,False,False
2020-08-28 11:41:37-04:00,349.12,349.13,300,2700,False,False
2020-08-28 11:41:37-04:00,349.12,349.13,300,2800,False,False
...,...,...,...,...,...,...
2020-08-28 11:41:40-04:00,349.10,349.12,2300,4100,False,False
2020-08-28 11:41:40-04:00,349.10,349.12,2300,4000,False,False
2020-08-28 11:41:40-04:00,349.10,349.12,2300,3800,False,False


,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-08-28 04:00:00,349.26,349.43,349.26,349.43,8,349.343,5
2020-08-28 04:05:00,349.48,349.48,349.43,349.43,26,349.435,7
2020-08-28 04:10:00,349.46,349.48,349.43,349.48,39,349.441,10
2020-08-28 04:15:00,349.55,349.79,349.55,349.79,126,349.657,49
2020-08-28 04:20:00,349.75,350.02,349.75,350.02,53,349.881,18
...,...,...,...,...,...,...,...
2020-08-28 19:35:00,350.94,350.96,350.92,350.95,18,350.950,8
2020-08-28 19:40:00,350.94,350.98,350.94,350.96,48,350.962,13
2020-08-28 19:45:00,350.96,350.96,350.96,350.96,13,350.960,5


,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-08-28 04:00:00,349.25,349.98,348.26,349.32,-1,-1.0,-1
2020-08-28 04:05:00,349.44,349.59,349.33,349.50,-1,-1.0,-1
2020-08-28 04:10:00,349.46,349.62,349.35,349.49,-1,-1.0,-1
2020-08-28 04:15:00,349.67,349.83,349.51,349.70,-1,-1.0,-1
2020-08-28 04:20:00,349.83,350.08,349.68,349.86,-1,-1.0,-1
...,...,...,...,...,...,...,...
2020-08-28 19:35:00,350.92,350.96,350.91,350.95,-1,-1.0,-1
2020-08-28 19:40:00,350.94,350.99,350.91,350.98,-1,-1.0,-1
2020-08-28 19:45:00,350.93,350.99,350.93,350.97,-1,-1.0,-1


,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-08-28 04:00:00,349.120,349.455,349.120,349.405,-1,-1.0,-1
2020-08-28 04:05:00,349.405,349.555,349.355,349.445,-1,-1.0,-1
2020-08-28 04:10:00,349.445,349.590,349.375,349.530,-1,-1.0,-1
2020-08-28 04:15:00,349.530,349.805,349.525,349.760,-1,-1.0,-1
2020-08-28 04:20:00,349.760,350.055,349.690,350.030,-1,-1.0,-1
...,...,...,...,...,...,...,...
2020-08-28 19:35:00,350.925,350.955,350.915,350.935,-1,-1.0,-1
2020-08-28 19:40:00,350.935,350.980,350.925,350.950,-1,-1.0,-1
2020-08-28 19:45:00,350.950,350.975,350.945,350.970,-1,-1.0,-1


In [3]:
from datetime import datetime as dt, timedelta as td
contract = Stock('SPY', 'SMART', 'USD')
# def getTicks(date):
date = dt.now()
start = ''
end = "20200828 11:41:41"

In [4]:
start = ''
end = "20200904 16:00:00"
contract = Stock('SPY', 'SMART', 'USD')
def qryHistTicks(contract,dtstr,nticks=1000,mode='end'):
    df = pd.DataFrame()
    if mode == 'end':
        start = ''
        end = f'{dtstr}'
    else:
        start = f'{dtstr}'
        end = ''
    try:
        ticks = ib.reqHistoricalTicks(contract, start, end, nticks, 'TRADES', useRth=False)
        df = util.df(ticks)
        df = df.set_index('time')
        df = df.tz_convert('America/Toronto')
        df['pastLimit'] = [row.pastLimit for row in df['tickAttribLast']]
        df['unreported'] = [row.unreported for row in df['tickAttribLast']]
        del df['tickAttribLast']
    except Exception as e: print(e)
    size = len(df)
    first = df.index[0]
    last = df.index[-1]
    nextm = None

    if mode == 'end': 
        display(first)
        nex = first + td(seconds=0)
        df = df[df.index != first]
    else: 
        nex = last - td(seconds=0)
        df = df[df.index != last]
    nextm = nex.strftime("%Y%m%d %H:%M:%S")
    
    return df,nextm

df_full = pd.DataFrame()
df,next_time = qryHistTicks(contract,"20200828 23:59:99",mode='end')
while next_time != None:
    print("going ",next_time)
    df,next_time = qryHistTicks(contract,next_time,mode='end')
    df_full = df_full.append(df)


Timestamp('2020-08-28 18:12:57-0400', tz='America/Toronto')

going  20200828 18:12:57


Timestamp('2020-08-28 17:03:18-0400', tz='America/Toronto')

going  20200828 17:03:18


Timestamp('2020-08-28 16:29:56-0400', tz='America/Toronto')

...

Timestamp('2020-08-28 16:06:23-0400', tz='America/Toronto')

going  20200828 16:06:23


Timestamp('2020-08-28 16:02:47-0400', tz='America/Toronto')

going  20200828 16:02:47


KeyboardInterrupt: 

In [6]:
df_full.to_csv('SPY_tape_sample.csv')

In [3]:
import asyncio, asyncpg, os, nest_asyncio
nest_asyncio.apply()

async def execSQL(qry):
    conn = await asyncpg.connect(user="postgres",password="FoolishPassword",database='stocks',host='postgres',port='5432',)
    await conn.execute(f'''{qry}''')
    await conn.close()

async def copyDF(df,schema,table):
    conn = await asyncpg.connect(user="postgres",password="FoolishPassword",database='stocks',host='postgres',port='5432',)
    await conn.execute(f'''SET search_path TO {schema};''')
    tuples = list(df.itertuples(index=True, name=None))
    li=list(df.columns)
    li.insert(0,df.index.name)
    display(tuples)
    await conn.copy_records_to_table(table, records=tuples, columns=li, timeout=10)
    await conn.close()



s = f'CREATE SCHEMA IF NOT EXISTS "GE" AUTHORIZATION postgres;'
asyncio.get_event_loop().run_until_complete(execSQL(s))

# s = await conn.copy_records_to_table(table_name, records=tuples, columns=list(df.columns), timeout=10)

In [84]:

display(li)

['date', 'open', 'high', 'low', 'close', 'volume', 'average', 'barcount']

In [23]:
end = "20200904 23:59:00"
histbarExample = ['1 D','5 secs','TRADES']
def histBar(endt,dStr,bSize,show,useRTH=False):
    bars = ib.reqHistoricalData(
        contract,endDateTime=end, durationStr=f'{dStr}',
        barSizeSetting=f'{bSize}', whatToShow=f'{show}', useRTH=useRTH)
    df = util.df(bars)
    df = df.set_index('date')
    df.columns = map(str.lower, df.columns)
    return df

df = histBar(end,histbarExample[0],histbarExample[1],histbarExample[2])
display(df)
df.to_csv('SPY_bar_sample_5sec.csv')
asyncio.get_event_loop().run_until_complete(copyDF(df,'spy','d1'))

,open,high,low,close,volume,average,barcount
date,,,,,,,
2020-09-04 04:00:00,347.00,347.00,346.86,346.89,81,346.989,4
2020-09-04 04:00:05,346.89,346.89,346.89,346.89,0,346.890,0
2020-09-04 04:00:10,346.89,346.89,346.89,346.89,0,346.890,0
2020-09-04 04:00:15,347.00,347.00,347.00,347.00,1,347.000,1
2020-09-04 04:00:20,347.00,347.00,347.00,347.00,0,347.000,0
...,...,...,...,...,...,...,...
2020-09-04 19:59:35,341.99,341.99,341.99,341.99,0,341.990,0
2020-09-04 19:59:40,342.00,342.00,342.00,342.00,1,342.000,1
2020-09-04 19:59:45,342.00,342.00,342.00,342.00,0,342.000,0


NameError: name 'asyncio' is not defined

In [32]:
end = "20200904 23:59:00"
stock,bss = "NVDA",'15 secs'
bss_trim = bss.replace(' ','')
contract = Stock(stock, 'SMART', 'USD')

he = ['1 D','5 secs','TRADES']
def histBar(endt,dStr,bSize,show,useRTH=False):
    bars = ib.reqHistoricalData(
        contract,endDateTime=end, durationStr=f'{dStr}',
        barSizeSetting=f'{bSize}', whatToShow=f'{show}', useRTH=useRTH)
    df = util.df(bars)
    df = df.set_index('date')
    df.columns = map(str.lower, df.columns)
    return df
df = histBar(end,he[0],bss,"TRADES",useRTH=False)

display(df)
df.to_csv(f'{stock}_bar_sample_{bss_trim}.csv')

,open,high,low,close,volume,average,barcount
date,,,,,,,
2020-09-04 04:00:00,512.00,512.00,512.00,512.00,20,512.000,2
2020-09-04 04:00:15,512.00,512.00,512.00,512.00,0,512.000,0
2020-09-04 04:00:30,512.98,512.98,512.98,512.98,1,512.980,1
2020-09-04 04:00:45,512.98,512.98,512.98,512.98,0,512.980,0
2020-09-04 04:01:00,513.87,513.90,513.87,513.90,2,513.882,2
...,...,...,...,...,...,...,...
2020-09-04 19:58:45,499.52,499.52,499.52,499.52,0,499.520,0
2020-09-04 19:59:00,499.52,499.52,499.52,499.52,0,499.520,0
2020-09-04 19:59:15,499.60,499.60,499.55,499.55,2,499.575,2
